In [1]:
%pip install -qU langchain langchain-core langchain-community faiss-cpu sentence-transformers pypdf
%pip install -qU litellm langchain-litellm
# 如用 Ollama（本地 LLM），请在系统里先安装并拉取模型：  https://ollama.com
# 示例：在终端运行  ollama pull llama3


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os, warnings, logging
warnings.filterwarnings("ignore")

# 关闭 LangSmith 追踪（避免未配置时报 401/告警）
os.environ["LANGCHAIN_TRACING_V2"] = "false"
for k in ["LANGCHAIN_API_KEY","LANGCHAIN_ENDPOINT","LANGCHAIN_PROJECT"]:
    os.environ.pop(k, None)

logging.getLogger("langchain").setLevel(logging.ERROR)
logging.getLogger("langsmith").setLevel(logging.ERROR)

# === 选择你的 LLM 方案（二选一） ===
USE_GITHUB_MODELS = True    # GitHub Models（免费额度，需 GITHUB_TOKEN）
USE_OLLAMA = False          # 本地 LLM（需本机安装 ollama 且已 pull 模型）

# GitHub Models 的 PAT（scopes: models 或 models:read）
# 建议在系统环境变量里配置 GITHUB_TOKEN；此处读取
os.environ["GITHUB_TOKEN"] = os.getenv("GITHUB_TOKEN", "")


In [3]:
from pathlib import Path
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# 你的数据目录（可放 txt/md/pdf）
DATA_DIR = Path("data/private_corpus")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# 若目录为空，给一个最小示例文件，方便直接跑通
if not any(DATA_DIR.iterdir()):
    (DATA_DIR/"sample.txt").write_text(
        "孔乙己是站着喝酒而穿长衫的唯一的人。他身材很高大；青白脸色，皱纹间时常夹些伤痕；"
        "他原来也读过书，但终于没有进学；写得一笔好字，便替人抄抄书，换一碗饭吃。",
        encoding="utf-8"
    )

def load_corpus(data_dir: Path):
    docs = []
    # txt / md
    for p in data_dir.rglob("*.txt"):
        docs += TextLoader(str(p), encoding="utf-8").load()
    for p in data_dir.rglob("*.md"):
        docs += TextLoader(str(p), encoding="utf-8").load()
    # pdf（可选）
    for p in data_dir.rglob("*.pdf"):
        docs += PyPDFLoader(str(p)).load()
    # 附加来源元信息
    for d in docs:
        d.metadata["source"] = d.metadata.get("source", d.metadata.get("file_path", "unknown"))
    return docs

raw_docs = load_corpus(DATA_DIR)
print(f"加载原始文档数：{len(raw_docs)}")

splitter = RecursiveCharacterTextSplitter(
    chunk_size=700, chunk_overlap=120,
    separators=["\n\n","\n","。","；","，"," ",""]
)
docs = splitter.split_documents(raw_docs)
for i, d in enumerate(docs):
    d.metadata["chunk_id"] = i

print(f"切分后文档块数：{len(docs)}")
print("示例片段：", docs[0].page_content[:120])


加载原始文档数：3
切分后文档块数：8
示例片段： 鲁镇的酒店的格局，是和别处不同的：都是当街一个曲尺形的大柜台，柜里面预备着热水，可以随时温酒。做工的人，傍午傍晚散了工，每每花四文铜钱，买一碗酒，——这是二十多年前的事，现在每碗要涨到十文，——靠柜外站着，热热的喝了休息；倘肯多花一文，便可


In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, emb)

# 可选：持久化
INDEX_DIR = "faiss_index_private"
vectorstore.save_local(INDEX_DIR)

retriever = vectorstore.as_retriever(search_kwargs={"k": 6})
print("FAISS 向量库就绪 ✅")


C:\Users\wangy\AppData\Local\Temp\ipykernel_21104\428255163.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


FAISS 向量库就绪 ✅


In [5]:
# 二选一的 LLM；其余逻辑不变
if USE_GITHUB_MODELS:
    from langchain_litellm import ChatLiteLLM
    GH_TOKEN = os.environ.get("GITHUB_TOKEN", "")
    if not GH_TOKEN:
        raise RuntimeError("未找到 GITHUB_TOKEN 环境变量。请设置后重试。")
    llm = ChatLiteLLM(
        model="openai/gpt-4o-mini",                 # 可换：openai/gpt-4o, openai/gpt-4.1-mini 等
        api_base="https://models.github.ai/inference",  # 关键：GitHub Models 的 /inference 端点
        api_key=GH_TOKEN,
        temperature=0,
    )
    print("LLM: GitHub Models / gpt-4o-mini")
elif USE_OLLAMA:
    from langchain_community.chat_models import ChatOllama
    llm = ChatOllama(model="llama3", temperature=0)     # 需本机已 ollama pull llama3
    print("LLM: Ollama / llama3")
else:
    raise RuntimeError("请至少启用一种 LLM 方案（GitHub Models 或 Ollama）。")


LLM: GitHub Models / gpt-4o-mini


In [6]:
from pathlib import Path
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import StrOutputParser

mq = MultiQueryRetriever.from_llm(
    retriever=retriever,
    llm=llm,
    include_original=True,  # 加入原始问题
)

SYSTEM_PROMPT = (
    "你是企业内知识助手。严格遵守："
    "1) 仅根据‘检索上下文’作答；若上下文无法支持，回答“我不确定”。"
    "2) 答案精炼分点。"
    "3) 末尾附引用列表，格式：[source: 文件名#chunk_id]。"
)

QA_PROMPT = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    MessagesPlaceholder("history"),
    ("human", "用户问题：{question}\n\n检索上下文：\n{context}\n\n请作答：")
])

def format_docs(docs):
    return "\n\n".join(
        f"[{d.metadata.get('chunk_id')}] ({Path(d.metadata.get('source','unknown')).name})\n{d.page_content}"
        for d in docs
    )

def citations(docs):
    outs = []
    for d in docs:
        src = Path(d.metadata.get("source","unknown")).name
        cid = d.metadata.get("chunk_id")
        outs.append(f"[source: {src}#{cid}]")
    # 去重并保持稳定顺序
    return list(dict.fromkeys(outs))


In [7]:
from pathlib import Path
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import StrOutputParser
from langchain_core.runnables import (
    RunnableParallel, RunnablePassthrough, RunnableLambda, RunnableWithMessageHistory
)
from langchain_core.chat_history import InMemoryChatMessageHistory

# ===== 会话历史（内存）=====
SESSION_STORE = {}
def get_session_history(session_id: str):
    if session_id not in SESSION_STORE:
        SESSION_STORE[session_id] = InMemoryChatMessageHistory()
    return SESSION_STORE[session_id]

def history_factory(session_id: str):
    return get_session_history(session_id)

# ===== Prompt =====
rewrite_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一名助手。基于对话历史，将用户问题改写为独立、具体、可检索的问题。只输出改写后的问题。"),
    MessagesPlaceholder("history"),
    ("human", "{question}"),
])
QA_PROMPT = ChatPromptTemplate.from_messages([
    ("system", "你是企业内知识助手。仅依据‘检索上下文’作答；无法确定就回答“我不确定”。答案精炼分点，末尾列出来源 [source: 文件名#chunk]。"),
    MessagesPlaceholder("history"),
    ("human", "用户问题：{question}\n\n检索上下文：\n{context}\n\n请作答："),
])

rewrite_chain = rewrite_prompt | llm | StrOutputParser()

def format_docs(docs):
    return "\n\n".join(
        f"[{d.metadata.get('chunk_id')}] ({Path(d.metadata.get('source','unknown')).name})\n{d.page_content}"
        for d in docs
    )

def citations(docs):
    outs = []
    for d in docs:
        src = Path(d.metadata.get("source","unknown")).name
        cid = d.metadata.get("chunk_id")
        outs.append(f"[source: {src}#{cid}]")
    # 去重并稳定顺序
    return " ".join(list(dict.fromkeys(outs)))

# ===== 构建 rag_core：改写 → 多查询召回 → 组装上下文/引用 → 回答 → 拼接引用 =====
rag_core = (
    RunnableParallel(
        # 原问题直接透传；history 需要显式取列表：x["history"]
        question = RunnablePassthrough(),
        history  = RunnableLambda(lambda x: x["history"]),
        # 改写需要 question + history
        rewritten_question = rewrite_chain,
    )
    # 1) 基于改写后的问题做 MultiQuery 召回
    | RunnableLambda(lambda x: {**x, "docs": mq.invoke(x["rewritten_question"])})
    # 2) 形成上下文与引用文本
    | RunnableLambda(lambda x: {**x, "context": format_docs(x["docs"]), "cites": citations(x["docs"])})
    # 3) 送入回答 Prompt
    | QA_PROMPT
    | llm
    | StrOutputParser()
    # 4) 与引用拼接
    | RunnableLambda(lambda answer, **kw: answer + (f"\n\n参考来源： {kw.get('cites','')}" if kw.get('cites') else ""))
).with_config(run_name="RAG-Core")

# ===== 包装成“带记忆”的链 =====
rag_with_memory = RunnableWithMessageHistory(
    rag_core,
    history_factory,                 # 只接收 session_id: str
    input_messages_key="question",   # 本轮输入的字段
    history_messages_key="history",  # 注入到两个 Prompt 的占位符
)

print("✅ RAG + 记忆 链已就绪")


✅ RAG + 记忆 链已就绪


In [8]:
cfg = {"configurable": {"session_id": "user-001"}}

print("Q1:")
print(
    rag_with_memory.invoke({"question": "根据语料，孔乙己的性格特点是什么？"}, config=cfg)
)

print("\nQ2（指代上一轮）：")
print(
    rag_with_memory.invoke({"question": "他有哪些典型行为可以作为证据？"}, config=cfg)
)


Q1:
孔乙己的性格特点包括：

1. **贫穷与落魄**：他曾经读过书，但未能进学，生活愈发贫困，甚至到了讨饭的地步。
2. **好喝懒做**：他有懒惰的习惯，常常不愿意工作，导致生活更加困窘。
3. **偷窃行为**：为了生存，他偶尔会偷窃，但在酒馆中品行相对较好，通常按时还清欠款。
4. **自尊心强**：尽管被人嘲笑，他仍然坚持自己的清白，认为“窃书不能算偷”。
5. **幽默与乐观**：他能带给周围人快乐，尽管生活艰难，仍能在酒馆中引发笑声。
6. **孤独与颓唐**：随着生活的恶化，他的形象逐渐颓废，最终可能死于贫困。

这些特点共同构成了孔乙己这一悲剧性人物的形象。  
[source: kongyiji.txt#1]

Q2（指代上一轮）：
孔乙己的典型行为可以作为证据的包括：

1. **偷窃**：他因生活困窘而偶尔偷窃，甚至偷到丁举人家里，最终被打折了腿。
2. **借酒消愁**：他常常在酒馆中喝酒，尽管欠债，仍然要求温酒，表现出对酒的依赖。
3. **自我辩解**：面对他人对偷窃的指责，他坚持认为“窃书不能算偷”，显示出他的自尊心和对知识的执着。
4. **与孩子互动**：他给邻居孩子豆子，表现出他仍有一丝善良和乐于助人的一面。
5. **颓唐与无助**：在被嘲笑时，他显得颓唐不安，反映出他内心的孤独与无助。

这些行为体现了孔乙己的复杂性格和悲剧命运。  
[source: kongyiji.txt#3]


In [ ]:
cfg2 = {"configurable": {"session_id": "user-002"}}

print("Q1:")
print(
    rag_with_memory.invoke({"question": "My name is Yongjun Wang, I am from China"}, config=cfg2)
)

print("\nQ2（指代上一轮）：")
print(
    rag_with_memory.invoke({"question": "What am I from?"}, config=cfg2)
)


Q1:
我不确定。 [source: 文件名#chunk]

Q2（指代上一轮）：
你来自中国。 [source: 文件名#chunk]
